# **Random Forest**
- Using data that was annotated
- Training it on the unbalanced data
- Using avgpool
- 1 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/AveragePooled/split_features_1s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[-0.01898465, -0.2811592 , -0.25845313, ..., -0.9981291 ,
         -0.9983617 , -1.        ],
        [-0.01670736, -0.27276382, -0.2651618 , ..., -0.9903572 ,
         -0.9974626 , -1.        ],
        [-0.01776626, -0.26800779, -0.26286083, ..., -0.9897578 ,
         -0.9985065 , -0.9999572 ],
        ...,
        [-0.07289265, -0.5997221 , -0.6325296 , ..., -0.9177227 ,
         -0.93674475, -0.99452364],
        [-0.01720924, -0.9334746 , -0.9393428 , ..., -0.8331133 ,
         -0.8683263 , -0.98977023],
        [-0.06566753, -0.6877171 , -0.70655054, ..., -0.8833477 ,
         -0.89263326, -0.9825436 ]], dtype=float32),
 'mfcc': array([[-9.9767429e-01,  5.7248688e-01, -2.8398493e-01, ...,
          1.3116055e-02,  6.1448671e-02,  3.6781926e-02],
        [-9.9636453e-01,  5.7445627e-01, -2.7706087e-01, ...,
          2.2723455e-02,  6.6032313e-02,  2.8396677e-02],
        [-1.0000000e+00,  5.6324875e-01, -2.5171626e-01, ...,
          3.5349183e-02,  6.71

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(12565,)

array([ 0, 19,  2,  1, 11, 17, 15,  2,  2,  2,  0,  1,  1, 12,  0],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(12565, 128)

array([-0.01743852, -0.6508722 , -0.6916726 , -0.63577497, -0.6325771 ,
       -0.693267  , -0.69924164, -0.7079409 , -0.73118097, -0.7504752 ,
       -0.78667325, -0.7978573 , -0.80476624, -0.8203451 , -0.7949259 ,
       -0.789861  , -0.8088392 , -0.82532144, -0.83248717, -0.80361825,
       -0.83153164, -0.85650146, -0.85409397, -0.85105276, -0.8633566 ,
       -0.8759206 , -0.8782255 , -0.8650762 , -0.8831901 , -0.88035285,
       -0.8797179 , -0.8726353 , -0.870042  , -0.86516607, -0.84496367,
       -0.82393944, -0.798188  , -0.7757094 , -0.76359135, -0.7636499 ,
       -0.7554068 , -0.7708649 , -0.7789858 , -0.7851823 , -0.8036501 ,
       -0.82371676, -0.8172532 , -0.7944738 , -0.7687108 , -0.7613045 ,
       -0.73363197, -0.7210863 , -0.73323107, -0.7296212 , -0.7292174 ,
       -0.73931384, -0.76318365, -0.7856355 , -0.7871114 , -0.7552131 ,
       -0.7135708 , -0.67738   , -0.6638213 , -0.6613151 , -0.6510076 ,
       -0.67116624, -0.7190023 , -0.7187137 , -0.65514815, -0.60

'mfcc'

(12565, 20)

...

(12565, 84)

array([-29.39905 , -28.323952, -30.232784, -29.47406 , -27.153494,
       -24.825844, -26.720036, -26.476421, -28.381443, -35.24852 ,
       -35.75719 , -36.161526, -38.84649 , -38.180325, -36.2371  ,
       -36.643337, -38.62381 , -40.752415, -39.763824, -33.694458,
       -34.624397, -36.676933, -29.563038, -31.875635, -36.143166,
       -39.61375 , -39.852917, -40.979412, -41.2093  , -41.830776,
       -41.238277, -40.43606 , -42.073097, -43.597797, -43.47115 ,
       -44.162884, -46.430286, -48.147423, -48.957855, -49.706734,
       -50.141304, -51.65557 , -51.007893, -49.478077, -49.370472,
       -50.613445, -52.21544 , -52.10996 , -50.735115, -53.456318,
       -54.24676 , -55.094383, -56.596664, -56.408787, -55.866055,
       -57.131966, -56.22503 , -54.807167, -53.670033, -52.506256,
       -51.82573 , -51.714436, -53.293312, -53.486908, -52.728363,
       -51.071804, -51.70296 , -52.312763, -51.750168, -50.660168,
       -46.681023, -46.31015 , -45.479893, -43.51856 , -41.098

'id'

(12565,)

'XC483906.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(3318,)

array([ 3,  2,  1,  0,  0,  8, 19,  2, 19,  5, 19, 15, 11,  7,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(3318, 128)

array([-0.01707026, -0.39656842, -0.4183283 , -0.45894244, -0.49384403,
       -0.51980877, -0.52839357, -0.5510067 , -0.58286625, -0.5928059 ,
       -0.6128816 , -0.61979336, -0.6288633 , -0.64968634, -0.6670257 ,
       -0.6761421 , -0.67730856, -0.6816009 , -0.67545867, -0.6647226 ,
       -0.6719049 , -0.6619738 , -0.6471278 , -0.652049  , -0.64146125,
       -0.64391303, -0.6518209 , -0.67124003, -0.71799076, -0.70881754,
       -0.7226164 , -0.73732513, -0.7245811 , -0.71309674, -0.7026587 ,
       -0.7268016 , -0.76876825, -0.75947165, -0.76661104, -0.7662666 ,
       -0.76978624, -0.7542365 , -0.7629837 , -0.7993014 , -0.8069032 ,
       -0.80255187, -0.80604035, -0.8206661 , -0.817362  , -0.81978494,
       -0.83180475, -0.81759435, -0.8207392 , -0.82689744, -0.83627135,
       -0.8303414 , -0.8460805 , -0.85512656, -0.8524952 , -0.84508985,
       -0.83099365, -0.8284293 , -0.8292149 , -0.83307105, -0.8172412 ,
       -0.82091004, -0.8238109 , -0.8224975 , -0.8299492 , -0.82

'mfcc'

(3318, 20)

...

(3318, 84)

array([-19.192596 , -18.357492 , -18.763315 , -19.366571 , -16.929043 ,
       -14.4525385, -14.049117 , -12.759987 , -14.402209 , -17.90481  ,
       -16.231672 , -15.364329 , -16.775997 , -17.590565 , -16.408525 ,
       -19.18176  , -19.740923 , -20.840792 , -21.245556 , -20.723444 ,
       -20.999773 , -25.08835  , -23.356092 , -25.212355 , -26.946945 ,
       -24.921827 , -25.974691 , -28.842213 , -28.154224 , -28.030458 ,
       -28.919643 , -29.377167 , -31.023067 , -31.180063 , -34.014633 ,
       -33.669846 , -34.84148  , -35.023224 , -36.85372  , -36.162834 ,
       -37.3165   , -37.896534 , -40.190365 , -40.017704 , -41.065468 ,
       -40.98337  , -41.63271  , -41.198174 , -40.882187 , -40.49316  ,
       -39.26483  , -39.1765   , -38.733685 , -39.23859  , -42.05581  ,
       -44.75164  , -46.034424 , -44.174923 , -45.603477 , -48.402534 ,
       -48.42529  , -48.000134 , -50.627438 , -51.769367 , -53.32704  ,
       -52.579865 , -53.69759  , -53.566704 , -54.655746 , -54.5

'id'

(3318,)

'XC428404.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 244)

In [18]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 244)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.625075346594334


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.65      0.63       455
           1       0.41      0.68      0.51       492
           2       0.80      0.57      0.67       889
           3       0.87      0.99      0.93       150
           4       0.57      0.06      0.11        67
           5       0.43      0.30      0.36        43
           6       0.23      0.29      0.25        24
           7       1.00      0.93      0.96        44
           8       0.53      0.54      0.53        50
           9       0.96      0.93      0.95       169
          10       0.74      0.79      0.76        53
          11       0.38      0.23      0.29        66
          12       0.78      0.68      0.73        59
          13       0.49      0.53      0.51        57
          14       0.39      0.63      0.48        38
          15       0.46      0.36      0.41       311
          16       0.88      0.96      0.92   

### **With MFCCs only**

In [22]:
training_features = train_features['mfcc']

training_features.shape

(12565, 20)

In [23]:
validation_features = val_features['mfcc']

validation_features.shape

(3318, 20)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5744424352019288


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.60      0.82      0.69       455
           1       0.34      0.67      0.45       492
           2       0.74      0.36      0.49       889
           3       0.89      0.99      0.94       150
           4       0.50      0.04      0.08        67
           5       0.00      0.00      0.00        43
           6       0.32      0.33      0.33        24
           7       0.91      0.89      0.90        44
           8       0.49      0.66      0.56        50
           9       0.89      0.93      0.91       169
          10       0.10      0.11      0.11        53
          11       0.55      0.24      0.34        66
          12       0.68      0.71      0.69        59
          13       0.55      0.61      0.58        57
          14       0.62      0.74      0.67        38
          15       0.55      0.34      0.42       311
          16       1.00      0.93      0.96   

### **With Mel-Spectrogram Only**

In [27]:
training_features = train_features['melspectrogram']

training_features.shape

(12565, 128)

In [28]:
validation_features = val_features['melspectrogram']

validation_features.shape

(3318, 128)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5717299578059072


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.61      0.77      0.68       455
           1       0.37      0.58      0.45       492
           2       0.74      0.45      0.56       889
           3       0.85      0.97      0.91       150
           4       0.33      0.09      0.14        67
           5       0.30      0.30      0.30        43
           6       0.15      0.25      0.18        24
           7       0.95      0.80      0.86        44
           8       0.61      0.54      0.57        50
           9       0.93      0.75      0.83       169
          10       0.45      0.58      0.51        53
          11       0.24      0.11      0.15        66
          12       0.56      0.53      0.54        59
          13       0.47      0.47      0.47        57
          14       0.27      0.66      0.38        38
          15       0.45      0.36      0.40       311
          16       0.90      0.96      0.93   

## **With Chroma Only**

In [32]:
training_features = train_features['chroma']

training_features.shape

(12565, 12)

In [33]:
validation_features = val_features['chroma']

validation_features.shape

(3318, 12)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3860759493670886


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.41      0.51      0.46       455
           1       0.24      0.44      0.31       492
           2       0.43      0.36      0.39       889
           3       0.55      0.57      0.56       150
           4       0.03      0.01      0.02        67
           5       0.13      0.09      0.11        43
           6       0.00      0.00      0.00        24
           7       0.55      0.41      0.47        44
           8       0.16      0.10      0.12        50
           9       0.81      0.72      0.76       169
          10       0.00      0.00      0.00        53
          11       0.12      0.09      0.10        66
          12       0.83      0.25      0.39        59
          13       0.15      0.12      0.14        57
          14       0.36      0.13      0.19        38
          15       0.42      0.43      0.42       311
          16       0.36      0.40      0.38   

## **With CQT Only**

In [37]:
training_features = train_features['cqt']

training_features.shape

(12565, 84)

In [38]:
validation_features = val_features['cqt']

validation_features.shape

(3318, 84)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.4629294755877034


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.41      0.47      0.44       455
           1       0.33      0.65      0.43       492
           2       0.74      0.38      0.50       889
           3       0.54      0.51      0.52       150
           4       0.00      0.00      0.00        67
           5       0.58      0.33      0.42        43
           6       0.11      0.17      0.13        24
           7       0.97      0.89      0.93        44
           8       0.31      0.50      0.38        50
           9       0.90      0.71      0.79       169
          10       0.30      0.51      0.38        53
          11       0.61      0.33      0.43        66
          12       0.46      0.69      0.55        59
          13       0.57      0.46      0.50        57
          14       0.26      0.24      0.25        38
          15       0.27      0.25      0.26       311
          16       0.72      0.93      0.81   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(12565, 148)

In [43]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(3318, 148)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5988547317661241


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.65      0.79      0.71       455
           1       0.38      0.61      0.47       492
           2       0.75      0.46      0.57       889
           3       0.91      1.00      0.95       150
           4       0.50      0.03      0.06        67
           5       0.25      0.16      0.20        43
           6       0.22      0.25      0.24        24
           7       0.97      0.89      0.93        44
           8       0.65      0.68      0.67        50
           9       0.98      0.88      0.93       169
          10       0.46      0.60      0.52        53
          11       0.43      0.14      0.21        66
          12       0.62      0.66      0.64        59
          13       0.45      0.54      0.49        57
          14       0.34      0.68      0.46        38
          15       0.45      0.38      0.41       311
          16       0.92      0.98      0.95   

## **With MFCCs and Chroma**

In [47]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(12565, 32)

In [48]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 32)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.622664255575648


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.63      0.78      0.70       455
           1       0.35      0.66      0.46       492
           2       0.75      0.48      0.59       889
           3       0.86      1.00      0.92       150
           4       1.00      0.03      0.06        67
           5       0.00      0.00      0.00        43
           6       0.30      0.33      0.31        24
           7       0.98      0.98      0.98        44
           8       0.54      0.58      0.56        50
           9       0.95      0.98      0.97       169
          10       0.46      0.47      0.47        53
          11       0.54      0.41      0.47        66
          12       0.80      0.69      0.75        59
          13       0.57      0.60      0.58        57
          14       0.65      0.63      0.64        38
          15       0.61      0.43      0.51       311
          16       0.96      0.93      0.94   

## **With MFCCs and CQT**

In [52]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(12565, 104)

In [53]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 104)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5406871609403255


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.54      0.49      0.52       455
           1       0.32      0.70      0.44       492
           2       0.77      0.40      0.52       889
           3       0.83      0.93      0.87       150
           4       0.00      0.00      0.00        67
           5       0.59      0.37      0.46        43
           6       0.14      0.21      0.17        24
           7       1.00      0.98      0.99        44
           8       0.53      0.68      0.60        50
           9       0.91      0.97      0.94       169
          10       0.32      0.43      0.37        53
          11       0.59      0.29      0.39        66
          12       0.61      0.71      0.66        59
          13       0.46      0.56      0.51        57
          14       0.59      0.50      0.54        38
          15       0.42      0.30      0.35       311
          16       0.77      0.91      0.83   

## **With Chroma and CQT**

In [57]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(12565, 96)

In [58]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 96)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5180831826401446


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.48      0.49      0.49       455
           1       0.34      0.66      0.45       492
           2       0.77      0.50      0.61       889
           3       0.66      0.74      0.70       150
           4       0.00      0.00      0.00        67
           5       0.55      0.26      0.35        43
           6       0.18      0.25      0.21        24
           7       0.93      0.93      0.93        44
           8       0.32      0.36      0.34        50
           9       0.89      0.78      0.83       169
          10       0.44      0.75      0.56        53
          11       0.33      0.32      0.32        66
          12       0.57      0.69      0.63        59
          13       0.36      0.32      0.34        57
          14       0.19      0.26      0.22        38
          15       0.28      0.19      0.22       311
          16       0.71      0.87      0.78   

## **With Mel-Spectrogram and Chroma**

In [62]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(12565, 140)

In [63]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(3318, 140)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6190476190476191


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.64      0.79      0.71       455
           1       0.41      0.60      0.49       492
           2       0.77      0.52      0.62       889
           3       0.82      0.97      0.89       150
           4       0.21      0.04      0.07        67
           5       0.44      0.47      0.45        43
           6       0.20      0.25      0.22        24
           7       0.97      0.89      0.93        44
           8       0.57      0.54      0.56        50
           9       0.95      0.89      0.92       169
          10       0.72      0.74      0.73        53
          11       0.45      0.20      0.27        66
          12       0.68      0.58      0.62        59
          13       0.57      0.56      0.57        57
          14       0.27      0.68      0.38        38
          15       0.47      0.40      0.43       311
          16       0.89      0.98      0.93   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(12565, 212)

In [68]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(3318, 212)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5952380952380952


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.55      0.63      0.59       455
           1       0.41      0.70      0.52       492
           2       0.81      0.52      0.63       889
           3       0.81      0.96      0.88       150
           4       0.23      0.04      0.07        67
           5       0.47      0.40      0.43        43
           6       0.15      0.25      0.19        24
           7       1.00      0.93      0.96        44
           8       0.59      0.54      0.56        50
           9       0.96      0.89      0.93       169
          10       0.68      0.60      0.64        53
          11       0.36      0.14      0.20        66
          12       0.81      0.66      0.73        59
          13       0.48      0.54      0.51        57
          14       0.29      0.50      0.37        38
          15       0.44      0.34      0.39       311
          16       0.85      0.95      0.90   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 232)

In [73]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 232)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6208559373116335


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.61      0.66      0.64       455
           1       0.41      0.67      0.51       492
           2       0.81      0.57      0.67       889
           3       0.86      0.99      0.92       150
           4       0.29      0.03      0.05        67
           5       0.38      0.19      0.25        43
           6       0.17      0.25      0.20        24
           7       1.00      0.93      0.96        44
           8       0.56      0.54      0.55        50
           9       0.97      0.92      0.95       169
          10       0.66      0.66      0.66        53
          11       0.46      0.17      0.24        66
          12       0.75      0.69      0.72        59
          13       0.47      0.56      0.51        57
          14       0.48      0.71      0.57        38
          15       0.48      0.37      0.42       311
          16       0.86      0.98      0.92   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(12565, 160)

In [78]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(3318, 160)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6389391199517782


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.67      0.83      0.74       455
           1       0.46      0.65      0.54       492
           2       0.79      0.54      0.64       889
           3       0.86      0.99      0.92       150
           4       0.62      0.07      0.13        67
           5       0.42      0.33      0.37        43
           6       0.19      0.21      0.20        24
           7       1.00      0.89      0.94        44
           8       0.57      0.54      0.56        50
           9       0.96      0.93      0.95       169
          10       0.63      0.74      0.68        53
          11       0.32      0.15      0.21        66
          12       0.76      0.69      0.73        59
          13       0.48      0.54      0.51        57
          14       0.24      0.71      0.36        38
          15       0.46      0.37      0.41       311
          16       0.89      0.98      0.93   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.625
3,chroma,1.0,0.386
8,chroma_cqt,1.0,0.518
4,cqt,1.0,0.463
2,melspectrogram,1.0,0.572
9,melspectrogram_chroma,1.0,0.619
12,melspectrogram_chroma_mfcc,1.0,0.639
10,melspectrogram_cqt,1.0,0.595
11,melspectrogram_cqt_mfcc,1.0,0.621
1,mfcc,1.0,0.574


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.62       0.92
3                    chroma      0.38       0.78
8                chroma_cqt      0.52       0.88
4                       cqt      0.47       0.86
2            melspectrogram      0.57       0.91
9     melspectrogram_chroma      0.62       0.92
10       melspectrogram_cqt      0.59       0.91
11  melspectrogram_cqt_mfcc      0.63       0.92
1                      mfcc      0.56       0.92
6               mfcc_chroma      0.62       0.93
7                  mfcc_cqt      0.54       0.89
5       mfcc_melspectrogram      0.59       0.91
